In [2]:
import math
from matplotlib import pyplot as plt
import numpy as np
from cmath import sqrt

# Ejercicio 3

Suponga que un viajante tiene que visitar $n$ ciudades en el menor tiempo posible. Considere una matriz $D$ de tamaño $n x n$ cuyos elementos $d_{pq}$ denotan la distancia entre cada par de ciudades $(p, q)$. Se define un recorrido como una trayectoria cerrada que visita cada ciudad una y sólo una vez (a excepción de la ciudad de partida, a la cual debe regresar). El problema consiste en hallar el recorrido de mínima longitud.

Implemente el algoritmo de sistema de hormigas y utilícelo para resolver el problema del agente viajero considerando los datos proporcionados en el archivo `gr17.csv`. Analice el efecto de la tasa de evaporación $(ρ)$ y de la cantidad de feromona depositada $(\tau)$ sobre los resultados de la búsqueda. Para esto último compare el desempeño del algoritmo empleando los métodos global, local y uniforme para depósito de feromonas. Realice varias corridas con cada configuración experimental y considere el tiempo de búsqueda y la longitud de los caminos encontrados como medidas para comparar el desempeño. Construya una tabla comparativa con los resultados obtenidos.

In [4]:
# Matriz de distancias entre ciudades
distanceMatrix = np.genfromtxt('gtp4datos/gr17.csv', delimiter=',')
# relleno la diagonal con infinito para que no haya división por cero
np.fill_diagonal(distanceMatrix, np.inf)

# Matriz de feromonas inicial (entre 0 y 1)
sigmaMatrix = np.ones(distanceMatrix.shape) * np.random.rand(distanceMatrix.shape[0], distanceMatrix.shape[1])

# Otros parámetros
NAnts = 1                       # Cantidad de hormigas
alpha = 2                       # Exponente de feromonas
beta = 2                        # Exponente del parámetro de costo
path = [0]                      # Las hormigas arrancan del nodo 0
visited = [0]                   # al inicio no han visitado lugares
lenPath = 0
prev = 0

# parámetro que considera el inverso de la distancia
nu = 1.0 / distanceMatrix

# TODO Repetir con un While
# Para cada hormiga
for ant in range(NAnts):

    sigmaVector = sigmaMatrix[prev]
    sigmaVector[visited] = 0
    # calculo las probabilidades a cada lugar
    prob_ = sigmaVector ** alpha * (nu[prev] ** beta)
    prob = prob_ / prob_.sum()
    # se selecciona el camino a tomar segun la probabilidad
    step = np.random.choice(range(len(distanceMatrix)), 1, p = prob)[0]
    path.append(step)
    prev = step
    visited.append(step)


range(0, 17)